In [1]:
import torch
from torch import nn
from transformers import AutoTokenizer

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [3]:
# Configuração do Tokenizador
checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [4]:
# Dados de Treino
dados_treino = [
    ("salada de alface e tomate com cenoura", 1),
    ("suco de laranja com maçã e morango", 0),
    ("espinafre refogado com alho", 1),
    ("salada de frutas com banana e uva", 0),
    ("brócolis ao forno com azeite", 1),
    ("mousse de manga e maracujá", 0)
]

In [5]:
# Modelo Transformer Encoder
class TransformerTextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # Encoder Layer
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim, 
            nhead=8, 
            dim_feedforward=256,
            dropout=0.3,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=2)
        
        self.fc = nn.Linear(embedding_dim, num_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids):
        # input_ids: [batch_size, seq_len]
        embedded = self.embedding(input_ids)
        encoded = self.transformer_encoder(embedded)
        
        # Pegamos a média da representação da frase (Global Average Pooling)
        sentence_rep = encoded.mean(dim=1)
        return self.fc(self.dropout(sentence_rep))

In [6]:
# Preparação e Treino
vocab_size = tokenizer.vocab_size
embedding_dim = 64 
modelo = TransformerTextClassifier(vocab_size, embedding_dim, num_classes=2)
modelo.to(device)
otimizador = torch.optim.Adam(modelo.parameters(), lr=0.003)
loss_function = nn.CrossEntropyLoss()

In [7]:
print("Iniciando treinamento...")


for epoch in range(101): 
    running_loss = 0.0
    
    for texto, label in dados_treino:
        otimizador.zero_grad()
        inputs = tokenizer(texto, return_tensors="pt", padding=True, truncation=True)
        
        # Movemos os tensores para a GPU
        input_ids = inputs['input_ids'].to(device) 
        label_tensor = torch.tensor([label]).to(device)
        
        outputs = modelo(input_ids)
        loss = loss_function(outputs, label_tensor)
        
        loss.backward()
        otimizador.step()
        running_loss += loss.item()
        
    if epoch % 10 == 0:
        print("training epoch: ", epoch)
        print("loss:", running_loss/len(dados_treino))

print("Treinamento concluído!")

Iniciando treinamento...
training epoch:  0
loss: 1.128295583029588
training epoch:  10
loss: 0.003871956568521758
training epoch:  20
loss: 0.0020730709608566635
training epoch:  30
loss: 0.00241025272407569
training epoch:  40
loss: 0.0005639585967098052
training epoch:  50
loss: 0.0025571495571057312
training epoch:  60
loss: 0.0005942116040387191
training epoch:  70
loss: 0.00037253204694328207
training epoch:  80
loss: 0.0003025217544442664
training epoch:  90
loss: 0.0002358145350929893
training epoch:  100
loss: 0.0003400501439803823
Treinamento concluído!


In [8]:
def prever(texto):
    modelo.eval()
    with torch.no_grad():
        # Move o input para o device correto
        input_ids = tokenizer(texto, return_tensors="pt")['input_ids'].to(device)
        
        output = modelo(input_ids)
        predicao = output.argmax(1).item()
        return "Vegetal" if predicao == 1 else "Fruta"

In [9]:
# Teste prático
print(f"Resultado: {prever('suco de morango')}")
print(f"Resultado: {prever('cenoura cozida')}")

Resultado: Fruta
Resultado: Vegetal
